In [1]:
pip install yfinance tensorflow scikit-learn matplotlib pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [3]:
start = datetime(2010, 1, 1)
end = datetime.now()

stock = "GOOG"

data = yf.download(stock, start, end)

data = data[['Open','High','Low','Close','Volume']]

data.dropna(inplace=True)

data.head()


C:\Users\sreed\AppData\Local\Temp\ipykernel_26544\95569184.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stock, start, end)
[*********************100%***********************]  1 of 1 completed


Price,Open,High,Low,Close,Volume
Ticker,GOOG,GOOG,GOOG,GOOG,GOOG
Date,,,,,
2010-01-04,15.498976,15.562262,15.431981,15.494032,78541293
2010-01-05,15.504661,15.520977,15.365234,15.425801,120638494
2010-01-06,15.472029,15.472029,14.989966,15.036936,159744526
2010-01-07,15.065118,15.079951,14.651037,14.686883,257533695
2010-01-08,14.634968,14.913082,14.563524,14.882675,189680313


In [4]:
scaler = MinMaxScaler()

scaled_data = scaler.fit_transform(data)


In [5]:
sequence_length = 100

X = []
y = []

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])
    y.append(scaled_data[i, 3])  # Close price index

X = np.array(X)
y = np.array(y)


In [6]:
split = int(len(X) * 0.7)

X_train = X[:split]
y_train = y[:split]

X_test = X[split:]
y_test = y[split:]

In [7]:
model = Sequential()

model.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.3))

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(64))
model.add(Dropout(0.3))

model.add(Dense(25))
model.add(Dense(1))

model.compile(
    optimizer='adam',
    loss='mean_squared_error'
)

model.summary()

C:\Users\sreed\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 100, 128)            │          68,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100, 128)            │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 25)                  │           1,625 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 251,251 (981.45 KB)

 Trainable params: 251,251 (981.45 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=30,
    validation_split=0.1,
    callbacks=[early_stop]
)


Epoch 1/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 42s 337ms/step - loss: 3.3360e-04 - val_loss: 1.6103e-04
Epoch 2/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 14s 183ms/step - loss: 9.2216e-05 - val_loss: 2.1222e-04
Epoch 3/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 208ms/step - loss: 6.4938e-05 - val_loss: 1.8048e-04
Epoch 4/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 17s 224ms/step - loss: 6.0701e-05 - val_loss: 1.0524e-04
Epoch 5/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 20s 210ms/step - loss: 5.7607e-05 - val_loss: 2.1877e-04
Epoch 6/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 19s 192ms/step - loss: 5.0501e-05 - val_loss: 3.1905e-04
Epoch 7/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 21s 203ms/step - loss: 5.6700e-05 - val_loss: 1.2404e-04
Epoch 8/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 194ms/step - loss: 5.4478e-05 - val_loss: 9.7036e-05
Epoch 9/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 18s 220ms/step - loss: 5.4017e-05 - val_loss: 1.5265e-04
Epoch 10/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 19s 206ms/step - loss: 4.8414e-05 - val_loss: 2.5569e-04
Epoch 11/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 19s 184ms/

In [9]:
model.save("Latest_stock_price_model.keras")



In [10]:
predictions = model.predict(X_test)


38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step  


In [11]:
dummy = np.zeros((len(predictions), 5))
dummy[:,3] = predictions.reshape(-1)

predicted_prices = scaler.inverse_transform(dummy)[:,3]

dummy2 = np.zeros((len(y_test), 5))
dummy2[:,3] = y_test.reshape(-1)

actual_prices = scaler.inverse_transform(dummy2)[:,3]


In [12]:
rmse = np.sqrt(mean_squared_error(actual_prices, predicted_prices))

mae = mean_absolute_error(actual_prices, predicted_prices)

accuracy = 100 - (mae / np.mean(actual_prices)) * 100

print("RMSE:", rmse)
print("MAE:", mae)
print("Accuracy:", accuracy, "%")


RMSE: 9.832806907121178
MAE: 6.331286727761682
Accuracy: 95.90840295671752 %


In [13]:
model.save("Latest_stock_price_model_v2.keras")

In [14]:
import os
print(os.getcwd())


C:\Users\sreed
